# 17 - Bag of Visual Words

Neste roteiro iremos explorar o método Bag of Visual Words para a determinação de similaridade entre imagens. Apesar de existirem métodos prontos no OpenCV iremos implementar partes deste método nós mesmos para fixar melhor os conceitos. 

**Aviso**: Ao pesquisar sobre este modelo a maioria dos recursos será sobre classificação de imagens, mas ele também é adequado para similaridade de imagens. 

O OpenCV já possui uma implementação de *Bag of Visual Words* fácil de usar, mas ela esconde detalhes demais do funcionamento do método. Como nosso objetivo é exercitar os conceitos por trás do modelo, criaremos nossa própria versão a partir de funções auxiliares disponibilizadas pelo OpenCV e scikit-learn. No projeto você pode (deve?) usar as funções do OpenCV diretamente. 

Nosso roteiro é dividido em duas grandes etapas. A primeira etapa, feita nas partes $0-4$ é o **treinamento** do nosso modelo. É nesta etapa que selecionamos um conjunto de imagens para criar nosso vocabulário e que determinamos quantos padrões visuais colocaremos no histograma. Isto é feito com base em um algoritmo de *agrupamento*, que encontre conjuntos de padrões visuais similares. Note que não existe um gabarito que defina qual deve ser a similaridade entre as imagens nem qual o resultado de uma busca. Por isso dizemos que este tipo de modelo é **não-supervisionado**, ou seja, toda informação usada está presente nas próprias imagens. O **treinamento** pode ser um processo lento pois, idealmente, ele é feito somente uma vez. Voltaremos nesse assunto na próxima semana, então você pode prosseguir mesmo que esta explicação não esteja 100% clara. 




A segunda etapa é a **aplicação** do modelo treinado para computar similaridades entre imagens. Nesta etapa computamos a representação em histograma de padrões visuais de uma imagem de busca e fazemos a comparação com as imagens do nosso banco de dados. Esta etapa deve ser muito rápida, pois o usuário espera uma resposta rápida de sistemas de busca.

# Parte 0 - Instalação do OpenCV e banco de imagens

Os trabalhos usando o *Bag of Words* costumam usar os métodos *SIFT* ou *SURF* para extração e descrição de pontos de interesse. Estes métodos são patenteados e para usá-los é necessário  instalar o pacote *opencv-contrib-python*, disponível via *pip*. Por alguma razão a versão mais atual não contém estes métodos, então precisamos instalar uma versão específica ligeiramente mais antiga. 

**Atenção**: é necessário desinstalar seu opencv antigo antes de instalar este pacote!

    > pip install opencv-contrib-python==3.4.0.12

Hoje iremos trabalhar com as funções usadas na atividade passsada. Vamos usar também o conjunto de imagens *Caltech101*, disponíveis [aqui](http://www.vision.caltech.edu/Image_Datasets/Caltech101/#Download). 

# Parte 1 - revisão

**Exercício**: crie uma função `def computa_descritores(img)` que recebe uma imagem e devolve os descritores dela. Você deve usar o método `SURF` neste exercício e, como na atividade anterior, pode ignorar as posições de cada ponto de interesse encontrado.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import namedtuple
import os, sys
from random import shuffle


def computa_descritores(img):
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(img, None)    
    return des

**Exercício**: crie uma função `def le_descritores_imagens(pastas, max_items=5)` que recebe uma lista de pastas como argumento, computa os descritores de cada imagem dentro desta pasta e retorna uma tupla contendo uma lista com o caminho todas as imagens analisadas e uma matriz contendo seus descritores. Verifique que a matriz retornada é uma matriz do `numpy` e não uma lista de listas. Para tornar nosso processamento mais rápido, processe somente as 5 primeiras imagens de cada pasta. 

**Dica**: se não souber como listar os arquivos de um diretório em Python busque pela função `os.listdir`.

In [6]:
def le_descritores_imagens(pastas, max_items = 5):
    list_Names  = []
    list_Matrix = []
    list_Imgs   = []
    Tuple = namedtuple('Tupla', 'listNames, matrix ')
    
    for pasta in pastas:
        dir_Name = "./proj1/101_ObjectCategories/" + pasta + '/'
        img_Name = os.listdir(dir_Name)
        #shuffle(img_Name)
            
        for i in range(max_items):
            if(img_Name[i][0]=="."):
                img_Name[i]=img_Name[i][2:]
            name = dir_Name + img_Name[i]
            list_Imgs.append(cv2.imread(name))
            list_Names.append(name)
            
    for img in list_Imgs:
        list_Matrix.append(computa_descritores(img))
    
    tup = Tuple(list_Names, np.concatenate(list_Matrix)) 
    return tup

A base de dados *Caltech101* está organizada em várias subpastas, cada uma contendo objetos de uma categoria específica. Vamos trabalhar inicialmente com as pastas `["faces", "garfield", "platypus", "nautilus", "elephant", "gerenuk"]`. Chame sua função para estas pastas e guarde os resultados obtidos. 

In [7]:
folder = ["Faces", "garfield", "platypus", "nautilus", "elephant", "gerenuk"]
descriptor = le_descritores_imagens(folder)

# Parte 2 - Criação do vocabulário e histograma

A criação do vocabulário de padrões visuais é feita usando um algoritmo de *clustering*. Este algoritmo identifica agrupamentos de padrões visuais e retorna, para cada agrupamento, um padrão representante (chamado de centróide por ele ser a média de todos os padrões do agrupamento). Grosso modo, cada centróide representa um conjunto de padrões muito similares, logo poderíamos substituí-los pelo centróide.

A biblioteca *scikit-learn* já possui uma implementação do algoritmo *KMeans* que pode ser usada para identificar estes padrões. Veja [neste link](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) a documentação e um exemplo de uso.

Nesta parte do roteiro iremos executar a segunda parte do algoritmo: iremos criar o vocabulário e uma função que cria o histograma de padrões visuais.

**Exercício**: crie uma função `def cria_vocabulario(descritores, sz=300)` que aplica o algoritmo acima e devolve uma tupla contendo a matriz com cada centróide em uma linha e o objeto `KMeans` criado. Note que a matriz deverá ter `sz` linhas (valor padrão 300) e o mesmo número de colunas dos descritores computados na parte anterior. Você deverá chamar esta função com a matriz de descritores criada na parte anterior.

Chamaremos esta tupla nas próximas funções de vocabulário.

In [4]:
from sklearn.cluster import KMeans
def cria_vocabulario(descritores, sz = 300):                

    Tuple = namedtuple('Tupla', 'matrix kmeans')
    kmeans = KMeans(n_clusters = sz, random_state=0).fit(descritores)
    tup = Tuple(kmeans.cluster_centers_ , kmeans)

    return tup
vocabulario = cria_vocabulario(descriptor.matrix)

**Exercício**: crie uma função `def representa_histograma(img, vocab)` que recebe uma imagem e um vocabulário e devolve um histograma que a represente. Se você estiver em dúvida como isto deve ser feito, consulte os slides desta aula.

**Dica**: o objeto `KMeans` criado na função anterior já possui uma função que calcula as distâncias até cada centróide. Consulte [sua documentação](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans.transform) para entender como usá-la. 

In [5]:
def representa_histograma(img, vocab):
    img = cv2.imread(img)
    desc = computa_descritores(img)
    
    dist = vocab.kmeans.predict(desc)
    
    list_freq = [0] * 300
    
    for i in dist:
        list_freq[i] += 1
   
    return list_freq

Teste sua função com alguma das imagens do *Caltech101*. 

In [8]:
representa_histograma(descriptor.listNames[0], vocabulario)

[1,
 3,
 8,
 0,
 3,
 5,
 5,
 1,
 2,
 13,
 2,
 1,
 3,
 4,
 9,
 9,
 6,
 6,
 4,
 0,
 2,
 0,
 12,
 8,
 8,
 6,
 6,
 5,
 0,
 0,
 5,
 2,
 11,
 7,
 4,
 3,
 4,
 2,
 0,
 2,
 0,
 8,
 1,
 0,
 5,
 9,
 6,
 5,
 15,
 2,
 1,
 5,
 2,
 0,
 0,
 9,
 9,
 9,
 0,
 5,
 3,
 3,
 3,
 2,
 5,
 3,
 2,
 10,
 47,
 4,
 0,
 4,
 2,
 4,
 1,
 2,
 0,
 9,
 29,
 3,
 2,
 1,
 5,
 13,
 4,
 3,
 0,
 0,
 3,
 2,
 7,
 14,
 4,
 1,
 0,
 12,
 1,
 5,
 7,
 0,
 5,
 13,
 1,
 0,
 1,
 5,
 6,
 1,
 9,
 0,
 2,
 41,
 4,
 10,
 3,
 5,
 1,
 3,
 0,
 7,
 13,
 4,
 1,
 4,
 6,
 6,
 1,
 0,
 3,
 5,
 4,
 5,
 0,
 2,
 4,
 4,
 1,
 5,
 4,
 1,
 4,
 11,
 2,
 0,
 5,
 3,
 1,
 1,
 2,
 5,
 4,
 3,
 6,
 37,
 8,
 2,
 4,
 4,
 0,
 4,
 3,
 12,
 5,
 1,
 9,
 1,
 5,
 5,
 1,
 7,
 2,
 2,
 5,
 2,
 3,
 1,
 2,
 3,
 2,
 4,
 0,
 8,
 1,
 4,
 1,
 4,
 8,
 9,
 0,
 2,
 0,
 4,
 14,
 7,
 5,
 2,
 5,
 0,
 0,
 7,
 3,
 5,
 4,
 4,
 7,
 2,
 8,
 3,
 2,
 4,
 7,
 2,
 16,
 0,
 0,
 2,
 13,
 5,
 2,
 2,
 7,
 1,
 7,
 7,
 4,
 5,
 3,
 3,
 3,
 4,
 3,
 1,
 2,
 4,
 4,
 0,
 5,
 1,
 3,
 3,
 7,
 11,
 3,
 1,
 1,

# Parte 3 - Similaridade entre imagens

Na última parte criamos uma função que representa uma imagem como um histograma de padrões visuais. Agora nos resta comparar duas imagens e criar um ranqueamento de quais imagens do banco de dados são mais parecidas com uma imagem de busca.

**Exercício**: Um método comum para comparação de histogramas é a utilização da distância $\chi^2$. Escreva abaixo sua fórmula e interprete-a.

https://stats.stackexchange.com/questions/184101/comparing-two-histograms-using-chi-square-distance

(1/2) (∑((xi−yi)**2)/(xi+yi))

**Exercício**: o OpenCV já possui uma função de comparação de histogramas. Qual é ela? Como usá-la para computar a distância $\chi^2$.

https://docs.opencv.org/3.1.0/d6/dc7/group__imgproc__hist.html#gga994f53817d621e2e4228fc646342d386aa88d6751fb2bb79e07aa8c8717fda881

cv2.HISTCMP_CHISQR


In [6]:
def chi(hist1,hist2):
    hist1 = np.array(hist1)
    hist2 = np.array(hist2)
    dist = 0.5* np.sum((hist1-hist2)**2/hist1)
    return dist

# Parte 4 - juntando tudo

Vamos agora juntar todas as partes criadas anteriormente na versão $0.5$ de nosso buscador. Para cada pasta processada na parte 1, 

1. escolha uma imagem que não foi usada na determinação do vocabulário
2. compute a distância dela para todas as imagens do vocabulário
3. Mostre as três imagens com menor distância.

Note que os passos acima incluem representar cada imagem do vocabulário como um histograma. Tome cuidado para não fazer isto mais de uma vez. 

Faça seu código de maneira modular, de maneira que para realizar a busca você apenas chame uma função. Você pode supor que esta função recebe como entrada qualquer objeto retornado pelas funções das etapas anteriores. 

**Exercício**: mostre, visualmente, os resultados de três buscas feitas com seu trabalho e comente os resultados

In [ ]:
# busca 1

In [ ]:
# busca 2

In [ ]:
# busca 3

*Comente seus resultados aqui*